In [ ]:
#빅데이터분석기사 6회 실기 2유형. 다중분류
#3개의 종을 분류한다.
#데이터셋

import pandas as pd

#X_train = pd.read_csv("C://Users//sam99//Desktop//컴퓨터공부//빅데이터분석기사//실기//Penguin_clf_X_train.csv")
#X_test = pd.read_csv("C://Users//sam99//Desktop//컴퓨터공부//빅데이터분석기사//실기//Penguin_clf_X_test.csv")
#y_train = pd.read_csv("C://Users//sam99//Desktop//컴퓨터공부//빅데이터분석기사//실기//Penguin_clf_X_train.csv")

# 훈련용 X 데이터
X_train = pd.DataFrame({
    'ID': [1, 2, 3, 4, 5, 6, 7],
    'island_bill': ['Dream', 'Torgersen', 'Biscoe', 'Biscoe', 'Dream', 'Torgersen', 'Biscoe'],
    'bill_depth_mm': [15.2, 13.4, 18.6, 17.0, 14.3, 16.8, 13.1],
    'flipper_length_mm': [198, 183, 205, 202, 200, 192, 229],
    'body_mass_g': [4814, 4820, 3948, 4553, 4447, 3926, 4665],
    'sex': ['FEMALE', 'FEMALE', 'FEMALE', 'MALE', 'MALE', 'FEMALE', 'MALE'],
    'year': [2008, 2008, 2007, 2009, 2009, 2009, 2007]
})

# 훈련용 y 데이터
y_train = pd.DataFrame({
    'ID': [1, 2, 3, 4, 5, 6, 7],
    'species': ['Gentoo', 'Chinstrap', 'Adelie', 'Gentoo', 'Adelie', 'Chinstrap', 'Adelie']
})

# 테스트용 X 데이터
X_test = pd.DataFrame({
    'ID': [8, 9, 10],
    'island_bill': ['Biscoe', 'Dream', 'Dream'],
    'bill_depth_mm': [19.2, 18.7, 13.3],
    'flipper_length_mm': [196, 205, 201],
    'body_mass_g': [4472, 3247, 4240],
    'sex': ['MALE', 'FEMALE', 'MALE'],
    'year': [2008, 2007, 2009]
})


In [10]:
#1. train ID제거

#X_train_id = X_train.pop("ID")
#X_test_id = X_test.pop("ID")
#X_test_id만 제출 시 사용한다. 

#결측치 확인
X_train.head()
y_train.value_counts()
X_test.head()
X_train.isnull().sum()
X_test.isnull().sum()

#2. 칼럼 분리
import numpy as np
cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist() #숫자 타입을 제외한 칼럼은 추출이 된다.
num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
print(cat_cols, num_cols) #연도는 연속형 변수로 보기 어렵다. 범주형으로 본다. 따라서 제거한다.
num_cols.remove('year')
num_cols

['island_bill', 'sex'] ['bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'year']


['bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

In [12]:
#3. 훈련 데이터와 테스트 데이터 분리한다.

from sklearn.model_selection import train_test_split
#층화추출을 시행한다.
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train['species'],
    stratify = y_train['species'],
    test_size = 0.3,
    random_state = 42
)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((4, 6), (3, 6), (4,), (3,))

In [ ]:
#4. 모형 만들기 (이진분류, 다중분류 가능)

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score #옵션(필수 아님.)
from sklearn.ensemble import RandomForestClassifier

#피처 엔지니어링 정의 (★필수)
column_transformer = ColumnTransformer([
    ("scaler", StandardScaler(), num_cols),
    ("one_hot_encoder", OneHotEncoder(), cat_cols)], 
    remainder= 'passthrough')

#모형 만들기 코드(★필수)
pipeline = Pipeline([
    ("preprocessing", column_transformer),
    ("classifier", RandomForestClassifier(max_depth=3,random_state=42))
    #7회에서는 회귀문제가 출제되었다.

])


# 학습(★필수)
pipeline.fit(X_tr, y_tr)


c:\Users\sam99\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  ['bill_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(),
                                                  ['island_bill', 'sex'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=3, random_state=42))])

In [34]:
 #옵션) 시험장에서 스코어 점수를 상세 확인하고 싶은 경우(생략 가능)
def get_score(model, X_tr, X_val, y_tr, y_val):
    tr_pred = model.predict(X_tr)
    val_pred = model.predict(X_val)
    tr_score = f1_score(y_tr, tr_pred, average = "macro")
    val_score = f1_score(y_val, val_pred, average = "macro")
    return f"train: {tr_score}, validation: {val_score}"
get_score(pipeline, X_tr, X_val, y_tr, y_val)

'train: 1.0, validation: 0.5555555555555555'

In [ ]:
# *제출해야 하는 코드(★필수★)
final_preds = pipeline.predict(X_test)
result = pd.DataFrame({
    "ID": X_test_id,
    "preds" : final_preds
})

result

result.to_csv("수헙번호32190344.csv", index=False)

In [38]:
df = pd.read_csv("C://Users//sam99//Desktop//컴퓨터공부//빅데이터분석기사//실기//수헙번호32190344.csv")
df

,ID,preds
0,8,Adelie
1,9,Adelie
2,10,Adelie
